<H1>LDA topic modelling 2000 --> 2020</H1>
<p> starting with the Gensim model, beneath is the sklearn model<p>

import the libraries needed for the gensim model:

In [1]:
#import re
import numpy as np
import pandas as pd
from pprint import pprint

import nltk

#Gensim
import gensim #environment
import gensim.corpora as corpora #corpus
from gensim.utils import simple_preprocess #document preprocess
from gensim.models import CoherenceModel #Coherence model to check the well functioning of the model

# spacy for lemmatization
import spacy #--> recalling for the nlp the english core

# Plotting tools
import pyLDAvis #model visualization environment
import pyLDAvis.gensim_models  # dedicated to gensim models
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import tqdm as t #to check for the loops.

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Utente\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use','https','co'])

In [4]:
df = pd.read_csv('abcnews-date-text.csv')
df.head(10)

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
5,20030219,ambitious olsson wins triple jump
6,20030219,antic delighted with record breaking barca
7,20030219,aussie qualifier stosur wastes four memphis match
8,20030219,aust addresses un security council over iraq
9,20030219,australia is locked into war timetable opp


In [5]:
appending2003 = {}
i = 0
for row in  df['publish_date']:
    if '2020' not in str(row):
        if '2003' in str(row):
            appending2003[row] = df['headline_text'][i]
            i = i +  1
            
appending2020 = {}
i = 0
for row in  df['publish_date']:
    if '2012' not in str(row):
        if '2020' in str(row) or '2021' in str(row):
            appending2020[row] = df['headline_text'][i]
            i = i +  1

In [6]:
print(appending2020)

{20200101: 'girl injured in head on highway crash', 20200102: 'patterson no show displays govts arrogance crean', 20200103: 'webb favourite for ladies masters', 20200104: 'drought impact may be more far reaching', 20200105: 'korean books atp date with top seed roddick', 20200106: 'scotland refuse to panic', 20200107: 'christmas island residents not aware of detention', 20200108: 'man u agreed deal with eriksson ferguson', 20200109: 'telstra punished after massive hong kong write down', 20200111: 'clp confident of winning lower house seat', 20200112: 'ntini stars as bangladesh felled for 108', 20200113: 'colombian rebels demand rescuers back off', 20200114: 'tight security to greet cricketers in zimbabwe', 20200115: 'hauritz replaces warne in world cup squad', 20200116: 'safety authority okays qantass use of contract', 20200117: 'condamine river flows at last', 20200118: 'investigation into sydney rail incident', 20200119: 'n korea fires missile into sea of japan', 20200120: 'two trucki

In [7]:
df2003 = pd.pandas.DataFrame.from_dict(appending2003,orient='index',columns=['headline_text'])
series2003 = df2003['headline_text'].to_list() 
df2020 = pd.pandas.DataFrame.from_dict(appending2020,orient='index',columns=['headline_text'])
series2020 = df2020['headline_text'].to_list() 

In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

words2003 = list(sent_to_words(series2003))
words2020 = list(sent_to_words(series2020))


In [9]:
# Build the bigram and trigram machines
bigram2003 = gensim.models.Phrases(words2003, min_count=3, threshold=50) # higher threshold fewer phrases.
trigram2003 = gensim.models.Phrases(bigram2003[words2003], threshold=50)  

# Build the bigram and trigram machines
bigram2020 = gensim.models.Phrases(words2020, min_count=3, threshold=50) # higher threshold fewer phrases.
trigram2020 = gensim.models.Phrases(bigram2020[words2020], threshold=50)

#da manuale

def ngram_mod(generator):
    return gensim.models.phrases.Phraser(generator)
    



In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts] #remove the stopwords
 
    
    
    
def make_bigrams2003(texts):
    '''create bigrams for each document in corpus'''
    return [ngram_mod(bigram2003)[doc] for doc in texts] 

def make_trigrams2003(texts):
    """create trigrams for each document in corpus based on the bigrams"""
    return [ngram_mod(trigram2003)[ngram_mod(bigram2003)[doc]] for doc in texts]

def make_bigrams2020(texts):
    '''create bigrams for each document in corpus'''
    return [ngram_mod(bigram2020)[doc] for doc in texts] 

def make_trigrams(texts):
    """create trigrams for each document in corpus based on the bigrams"""
    return [ngram_mod(trigram2020)[ngram_mod(bigram2020)[doc]] for doc in texts]


#python -m spacy download en_core_web_sm since en does not word anymore
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
words_stoprem2003 = remove_stopwords(words2003)
gram2_2003 = make_bigrams2003(words_stoprem2003)

words_stoprem2020 = remove_stopwords(words2020)
gram2_2020 = make_bigrams2020(words_stoprem2020)

#define tags and lemmatize
data_lemmatized2003 = lemmatization(gram2_2003, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized2020 = lemmatization(gram2_2020, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [12]:
# Create Dictionary with the training data
i2w2003 = corpora.Dictionary(data_lemmatized2003)

# Create Corpus with the training data
texts2003 = data_lemmatized2003

# TDF of the train data
corpus2003 = [i2w2003.doc2bow(text) for text in texts2003]


In [13]:
# Create Dictionary with the training data
i2w2020 = corpora.Dictionary(data_lemmatized2020)

# Create Corpus with the training data
texts2020 = data_lemmatized2020

# TDF of the train data
corpus2020 = [i2w2020.doc2bow(text) for text in texts2020]

In [14]:
from gensim.models import ldamulticore
num_topics = list(range(1,15 )) #for test reasons we will have it up to 3 topiucs --> originally it is up to 16
num_keywords = 14 #change back to 15
i = 1
LDA_models2003 = {}
LDA_topics2003 = {}

for i in t.tqdm(num_topics):
    LDA_models2003[i] = ldamulticore.LdaMulticore(workers=3,corpus=corpus2003,
                             id2word=i2w2003,
                             num_topics=i,
                             chunksize=100,
                             passes=20,
                             random_state=42)
    

    shown_topics2003 = LDA_models2003[i].show_topics(num_topics=i, 
                                             num_words=num_keywords,
                                             formatted=False)
    LDA_topics2003[i] = [[word[0] for word in topic[1]] for topic in shown_topics2003]

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:26<00:00,  1.86s/it]


In [15]:
from gensim.models import ldamulticore
num_topics = list(range(1,15 )) #for test reasons we will have it up to 3 topiucs --> originally it is up to 16
num_keywords = 14 #change back to 15
i = 1
LDA_models2020 = {}
LDA_topics2020 = {}
perplexity_tot_test = {}
for i in t.tqdm(num_topics):
    LDA_models2020[i] = ldamulticore.LdaMulticore(workers=3,corpus=corpus2020,
                             id2word=i2w2020,
                             num_topics=i,
                             chunksize=100,
                             passes=20,
                             random_state=42)
    

    shown_topics2020 = LDA_models2020[i].show_topics(num_topics=i, 
                                             num_words=num_keywords,
                                             formatted=False)
    LDA_topics2020[i] = [[word[0] for word in topic[1]] for topic in shown_topics2020]

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:30<00:00,  2.20s/it]


In [16]:
coherences2003 = [CoherenceModel(model=LDA_models2003[i], texts=data_lemmatized2003, dictionary=i2w2003, coherence='c_v').get_coherence()\
              for i in t.tqdm(num_topics[:-1])]

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:25<00:00,  1.92s/it]


In [17]:
coherences2020 = [CoherenceModel(model=LDA_models2020[i], texts=data_lemmatized2020, dictionary=i2w2020, coherence='c_v').get_coherence()\
              for i in t.tqdm(num_topics[:-1])]

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:23<00:00,  1.81s/it]


In [18]:
print(coherences2003,coherences2020)

[0.6961513333734143, 0.630069150231438, 0.6714639880385249, 0.6563790504893541, 0.5717494561740015, 0.5651583172128442, 0.5842999395446568, 0.5755566365310795, 0.6000705824106481, 0.600158790879937, 0.5761347902647581, 0.5438324032441496, 0.5561545513524043] [0.7912036878996378, 0.7397175928725891, 0.6992376231616338, 0.6103027691300482, 0.5904108908678196, 0.6145917366625274, 0.5032729500911427, 0.5104224902833805, 0.5712993862404718, 0.5127669822476694, 0.4769271829689768, 0.4978319864647602, 0.4903797806937803]


In [19]:
pyLDAvis.enable_notebook()
vis2003 = pyLDAvis.gensim_models.prepare(LDA_models2003[3], corpus2003, i2w2003)
vis2003

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.117315 -0.004252       1        1  41.583553
0      0.062528 -0.094559       2        1  30.960502
2      0.054788  0.098812       3        1  27.455945, topic_info=         Term       Freq      Total Category  logprob  loglift
8    zimbabwe  43.000000  43.000000  Default  30.0000  30.0000
503      face   6.000000   6.000000  Default  29.0000  29.0000
27        zim   4.000000   4.000000  Default  28.0000  28.0000
352       new   5.000000   5.000000  Default  27.0000  27.0000
86      world   9.000000   9.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
17     worker   1.822337   8.066894   Topic3  -5.4763  -0.1951
109  adelaide   1.326708   2.247531   Topic3  -5.7937   0.7655
66     yemeni   1.304901   2.195367   Topic3  -5.8103   0.7724
34       save   1.302709   2.195892   Topic3  -5.8119   0.7704
450     leave   1.303230   2.248318   Topic3  -5.8115   0.7473

[158 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
109       1  0.444933   adelaide
109       3  0.444933   adelaide
908       1  0.920603      alice
44        1  0.997293     arrest
928       1  0.920607     arrive
...     ...       ...        ...
8         1  0.717078   zimbabwe
8         2  0.277578   zimbabwe
36        1  0.991891  zimbabwes
266       1  0.131934        zoo
266       2  0.791604        zoo

[155 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

In [20]:
vis2020 = pyLDAvis.gensim_models.prepare(LDA_models2020[3], corpus2020, i2w2020)
vis2020

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.050169  0.110482       1        1  34.549124
2     -0.141676 -0.019586       2        1  32.935244
0      0.091507 -0.090896       3        1  32.515631, topic_info=         Term       Freq      Total Category  logprob  loglift
157      govt  13.000000  13.000000  Default  30.0000  30.0000
375   service   8.000000   8.000000  Default  29.0000  29.0000
193      call  13.000000  13.000000  Default  28.0000  28.0000
1029      cat   5.000000   5.000000  Default  27.0000  27.0000
683    victim   5.000000   5.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
1045     road   2.832046   3.226252   Topic3  -5.2257   0.9931
330    defend   3.449149   3.948480   Topic3  -5.0286   0.9882
193      call   8.729306  13.125352   Topic3  -4.1000   0.7156
122     title   3.649853   4.971863   Topic3  -4.9720   0.8143
68      world   3.504819   9.954968   Topic3  -5.0126   0.0795

[131 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
705       1  0.792661      act
1023      1  0.883593     aged
1037      1  0.883590  alleged
237       1  0.958722      amp
1083      1  0.883593     apec
...     ...       ...      ...
49        2  0.920751      win
109       1  0.792660    woman
68        1  0.401809    world
68        2  0.301357    world
68        3  0.401809    world

[108 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])